# QAOA for QS/QV Prediction
## Dr. Jack Hammer - QMLE Director

**Quantum Approximate Optimization Algorithm** for predicting least-likely QV column activations.

### Key Features:
- **XY-Mixer**: Preserves Hamming weight (matches 5-activation constraint)
- **Cost Hamiltonian**: Encodes inverse frequency + anti-co-occurrence
- **GPU Acceleration**: PennyLane lightning.gpu with cuQuantum

### Hardware Requirements:
- RunPod 2x H200 GPUs (or equivalent)
- 39 qubits = 2^39 states = 4TB state vector (GPU distributed)

---

## 0. Environment Setup

In [ ]:
# Install requirements if needed (skip if already installed)
# !pip install -q pennylane pennylane-lightning pennylane-lightning-gpu
# !pip install -q numpy scipy pandas matplotlib seaborn tqdm

In [ ]:
import os
import sys
import numpy as np
import pennylane as qml
from datetime import datetime

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

print(f"PennyLane version: {qml.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Check GPU availability
try:
    import cupy as cp
    print(f"CuPy available: {cp.cuda.runtime.getDeviceCount()} GPU(s)")
except ImportError:
    print("CuPy not available - will use CPU fallback if needed")

# Check PennyLane devices
print("\nAvailable PennyLane devices:")
for plugin in ['lightning.qubit', 'lightning.gpu', 'default.qubit']:
    try:
        dev = qml.device(plugin, wires=2)
        print(f"  ✓ {plugin}")
    except Exception as e:
        print(f"  ✗ {plugin}: {e}")

## 1. Load Modules and Configuration

In [ ]:
from src import (
    QAOAConfig,
    QAOA_CONFIG,
    PENNYLANE_CONFIG,
    load_dataset,
    prepare_qaoa_data,
    compute_hamiltonian_weights,
    create_cost_hamiltonian,
    create_qaoa_circuit,
    create_qaoa_qnode,
    create_sampling_qnode,
    get_initial_params,
    QAOAOptimizer,
    run_optimization,
    samples_to_predictions,
    evaluate_prediction,
    generate_provenance,
    save_results,
    print_prediction_summary
)

print("All modules loaded successfully!")

In [ ]:
# Configuration
config = QAOAConfig(
    data_path='data/c5_Matrix.csv',
    n_qubits=39,
    n_layers=8,           # QAOA depth (p parameter)
    mixer_type='xy',      # XY-mixer for Hamming weight preservation
    n_select=20,          # Number of columns to predict
    w_freq=1.0,           # Weight for frequency term in Hamiltonian
    w_cooc=0.5,           # Weight for co-occurrence penalty
    optimizer='COBYLA',   # Derivative-free optimizer
    max_iterations=500,   # Max optimization steps
    n_shots=8192,         # Measurement shots
    seed=42
)

print("QAOA Configuration:")
print(f"  Qubits: {config.n_qubits}")
print(f"  Layers: {config.n_layers}")
print(f"  Mixer: {config.mixer_type}")
print(f"  Shots: {config.n_shots}")
print(f"  Optimizer: {config.optimizer}")

## 2. Load Dataset and Compute Hamiltonian Weights

In [ ]:
# Load and analyze dataset
qaoa_data = prepare_qaoa_data(config)

print(f"\nDataset loaded successfully!")
print(f"  Events: {qaoa_data['stats']['n_events']}")
print(f"  Columns: {qaoa_data['stats']['n_columns']}")
print(f"  Sparsity: {qaoa_data['stats']['sparsity']:.2%}")

In [ ]:
# Visualize column frequencies
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Column frequency distribution
ax1 = axes[0]
freqs = qaoa_data['stats']['column_frequencies']
ax1.bar(range(len(freqs)), freqs, color='steelblue')
ax1.axhline(y=freqs.mean(), color='red', linestyle='--', label=f'Mean: {freqs.mean():.4f}')
ax1.set_xlabel('Column Index (QV)')
ax1.set_ylabel('Activation Frequency')
ax1.set_title('QV Column Activation Frequencies')
ax1.legend()

# Single-qubit Hamiltonian weights (h)
ax2 = axes[1]
h = qaoa_data['h']
ax2.bar(range(len(h)), h, color='darkgreen')
ax2.set_xlabel('Column Index (QV)')
ax2.set_ylabel('h_i (Selection Bias)')
ax2.set_title('Hamiltonian Single-Qubit Weights\n(Higher = More Desirable)')

plt.tight_layout()
plt.savefig('results/hamiltonian_weights.png', dpi=150)
plt.show()

In [ ]:
# Visualize co-occurrence matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Raw co-occurrence
ax1 = axes[0]
sns.heatmap(qaoa_data['stats']['cooccurrence'], ax=ax1, cmap='Blues')
ax1.set_title('Co-occurrence Matrix')
ax1.set_xlabel('Column j')
ax1.set_ylabel('Column i')

# Hamiltonian J matrix
ax2 = axes[1]
sns.heatmap(qaoa_data['J'], ax=ax2, cmap='RdBu_r', center=0)
ax2.set_title('Hamiltonian Coupling Matrix (J)\n(Negative = Penalizes Co-selection)')
ax2.set_xlabel('Column j')
ax2.set_ylabel('Column i')

plt.tight_layout()
plt.savefig('results/coupling_matrix.png', dpi=150)
plt.show()

## 3. Create QAOA Circuit

The QAOA circuit alternates between:
1. **Cost Layer**: $U_C(\gamma) = e^{-i\gamma H_C}$ encodes the optimization objective
2. **Mixer Layer**: $U_B(\beta) = e^{-i\beta H_B}$ explores the solution space

The XY-mixer preserves Hamming weight, enforcing the 5-activation constraint naturally.

In [ ]:
# Select device based on availability
device_name = 'lightning.gpu'  # Use GPU if available

try:
    # Test with small circuit first
    test_dev = qml.device(device_name, wires=4)
    print(f"Using device: {device_name}")
except Exception as e:
    device_name = 'default.qubit'
    print(f"Falling back to: {device_name}")
    print(f"Warning: CPU simulation of 39 qubits will be extremely slow!")

In [ ]:
# Create QAOA QNode for optimization
print("Creating QAOA circuit...")

qaoa_qnode = create_qaoa_qnode(
    n_qubits=config.n_qubits,
    n_layers=config.n_layers,
    h=qaoa_data['h'],
    J=qaoa_data['J'],
    mixer_type=config.mixer_type,
    device_name=device_name,
    n_shots=config.n_shots
)

print(f"QAOA circuit created with {config.n_layers} layers")
print(f"Total parameters: {2 * config.n_layers} (gamma + beta per layer)")

## 4. Initialize Parameters

In [ ]:
# Initialize QAOA parameters
initial_params = get_initial_params(config.n_layers, seed=config.seed)

print(f"Initial parameters:")
print(f"  Shape: {initial_params.shape}")
print(f"  Gammas: {initial_params[:config.n_layers]}")
print(f"  Betas: {initial_params[config.n_layers:]}")

In [ ]:
# Test circuit with initial parameters
print("Testing circuit with initial parameters...")
initial_cost = qaoa_qnode(initial_params)
print(f"Initial cost: {initial_cost:.6f}")

## 5. Run QAOA Optimization

Using COBYLA optimizer (derivative-free, robust to noise).

In [ ]:
# Run optimization
print("Starting QAOA optimization...")
print("This may take several minutes on GPU (or hours on CPU)...")
print()

optimization_results = run_optimization(
    qaoa_qnode,
    initial_params,
    optimizer=config.optimizer,
    max_iterations=config.max_iterations,
    seed=config.seed
)

In [ ]:
# Plot optimization history
fig, ax = plt.subplots(figsize=(10, 5))

costs = optimization_results['history']['costs']
ax.plot(costs, 'b-', alpha=0.7)
ax.axhline(y=optimization_results['optimal_cost'], color='r', linestyle='--', 
           label=f'Optimal: {optimization_results["optimal_cost"]:.6f}')
ax.set_xlabel('Iteration')
ax.set_ylabel('Cost (negative expectation)')
ax.set_title('QAOA Optimization Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/optimization_history.png', dpi=150)
plt.show()

print(f"\nOptimization Summary:")
print(f"  Optimal cost: {optimization_results['optimal_cost']:.6f}")
print(f"  Iterations: {optimization_results['history']['iterations']}")
print(f"  Time elapsed: {optimization_results['elapsed_time']:.1f}s")

## 6. Sample from Optimized Circuit

In [ ]:
# Create sampling QNode
print("Creating sampling circuit...")

sampling_qnode = create_sampling_qnode(
    n_qubits=config.n_qubits,
    n_layers=config.n_layers,
    h=qaoa_data['h'],
    J=qaoa_data['J'],
    mixer_type=config.mixer_type,
    device_name=device_name,
    n_shots=config.n_shots
)

In [ ]:
# Sample from optimized circuit
print(f"Sampling {config.n_shots} measurements from optimized circuit...")

optimal_params = optimization_results['optimal_params']
samples = sampling_qnode(optimal_params)

print(f"Collected {len(samples)} samples")
print(f"Sample shape: {samples.shape}")

## 7. Process Predictions

In [ ]:
# Convert samples to predictions
predictions, analysis = samples_to_predictions(samples, n_select=config.n_select)

print(f"\nSample Analysis:")
print(f"  Mean Hamming weight: {analysis['mean_hamming_weight']:.2f}")
print(f"  Hamming weight distribution: {analysis['hamming_weight_distribution']}")

In [ ]:
# Visualize column selection frequencies
fig, ax = plt.subplots(figsize=(12, 5))

freqs = analysis['column_frequencies']
colors = ['darkred' if i in predictions else 'steelblue' for i in range(len(freqs))]

ax.bar(range(len(freqs)), freqs, color=colors)
ax.set_xlabel('Column Index (QV)')
ax.set_ylabel('QAOA Selection Frequency')
ax.set_title('QAOA Column Selection Frequencies\n(Red = Top 20 Predictions)')

plt.tight_layout()
plt.savefig('results/qaoa_selection_frequencies.png', dpi=150)
plt.show()

## 8. Evaluate and Compare to Baseline

In [ ]:
# Evaluate predictions
evaluation = evaluate_prediction(
    predictions,
    baseline_ranking=qaoa_data['baseline_ranking']
)

# Print summary
print_prediction_summary(predictions, analysis, evaluation)

In [ ]:
# Side-by-side comparison
print("\n" + "=" * 70)
print("COMPARISON: QAOA vs Baseline 1")
print("=" * 70)

baseline_cols = [f"QV_{i+1}" for i in qaoa_data['baseline_ranking']]
qaoa_cols = [f"QV_{i+1}" for i in predictions]

print(f"\n{'Rank':<6} {'Baseline 1':<15} {'QAOA':<15} {'Match':<10}")
print("-" * 50)

for rank in range(config.n_select):
    b_col = baseline_cols[rank]
    q_col = qaoa_cols[rank]
    match = "✓" if b_col == q_col else ""
    print(f"{rank+1:<6} {b_col:<15} {q_col:<15} {match:<10}")

print("-" * 50)
print(f"Agreement: {evaluation.get('overlap_with_baseline', 0)}/{config.n_select} = {evaluation.get('baseline_agreement_pct', 0):.1%}")

## 9. Save Results and Generate Provenance

In [ ]:
# Generate provenance
provenance = generate_provenance(
    config,
    qaoa_data['stats'],
    optimization_results,
    predictions,
    analysis
)

print("Provenance generated!")
print(f"  Experiment ID: {provenance['experiment_id']}")
print(f"  Result hash: {provenance['result_hash_sha512'][:64]}...")

In [ ]:
# Save all results
os.makedirs('results', exist_ok=True)

results_path = save_results(
    provenance,
    optimization_results,
    analysis,
    output_dir='results'
)

## 10. Final Summary

In [ ]:
print("\n" + "=" * 70)
print("QAOA EXPERIMENT COMPLETE")
print("Dr. Jack Hammer - QMLE Director")
print("=" * 70)

print(f"\nExperiment Summary:")
print(f"  Timestamp: {provenance['timestamp']}")
print(f"  Dataset hash: {provenance['dataset']['hash_sha512'][:32]}...")
print(f"  Result hash: {provenance['result_hash_sha512'][:32]}...")

print(f"\nQAOA Configuration:")
print(f"  Qubits: {config.n_qubits}")
print(f"  Layers: {config.n_layers}")
print(f"  Mixer: {config.mixer_type}")
print(f"  Optimizer: {config.optimizer}")
print(f"  Iterations: {optimization_results['history']['iterations']}")
print(f"  Time: {optimization_results['elapsed_time']:.1f}s")

print(f"\nPredictions:")
print(f"  Top 20 columns: {', '.join([f'QV_{i+1}' for i in predictions])}")
print(f"  Baseline agreement: {evaluation.get('overlap_with_baseline', 0)}/20")

print(f"\nResults saved to: results/")
print("=" * 70)